In [ ]:
%env  WORKDIR=/tmp/vault

### Connect to Vault-pr K8S cluster

In [ ]:
! terraform output -state=../1_Platform_Deployment/terraform.tfstate -raw configure_kubectl_cluster_pr

In [ ]:
! eval $(terraform output -state=../1_Platform_Deployment/terraform.tfstate -raw configure_kubectl_cluster_pr)

### Create Account for PODs

In [ ]:
! kubectl create sa internal-app

### Configure Vault

In [ ]:
# In a terminal

kubectl exec -n vault vault-pr-0  -i -t -- /bin/sh
---
# Log as user peter using LDAP Auth method
vault login -method=ldap username=peter

# Create a Local k8S Auth method
vault auth enable -path=k8snorth -local kubernetes

# Connect to local kubernetes cluster
vault write auth/k8snorth/config \
    kubernetes_host=https://$KUBERNETES_SERVICE_HOST:$KUBERNETES_SERVICE_PORT

# Create a role
vault write auth/k8snorth/role/internal-app \
    bound_service_account_names=internal-app \
    bound_service_account_namespaces=default \
    policies=internal-app \
    alias_name_source=serviceaccount_name \
    ttl=24h

# Create policy
vault policy write internal-app - <<EOF
path "internal/data/database/config" {
  capabilities = ["read"]
}
EOF

# Create a KVv2 engine
vault secrets enable -path=internal kv-v2

# Create Secret
vault kv put internal/database/config username="db-readonly-username" password="db-secret-password"

## Deploy POD with Vault Agent (sidecar) via annotations

In [ ]:
%%bash
cat > $WORKDIR/deployment-orgchart.yaml <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: orgchart
  labels:
    app: orgchart
spec:
  selector:
    matchLabels:
      app: orgchart
  replicas: 1
  template:
    metadata:
      annotations:
        vault.hashicorp.com/agent-inject: 'true'
        vault.hashicorp.com/role: 'internal-app'
        vault.hashicorp.com/agent-inject-secret-database-config.txt: 'internal/data/database/config'
        vault.hashicorp.com/tls-skip-verify: 'true' # Untrusted cert used here
      labels:
        app: orgchart
    spec:
      serviceAccountName: internal-app
      containers:
        - name: orgchart
          image: nginx
EOF

In [ ]:
%%bash
kubectl delete --filename $WORKDIR/deployment-orgchart.yaml
kubectl apply --filename $WORKDIR/deployment-orgchart.yaml

### Vault Agents Logs

In [ ]:
%%bash 
kubectl logs \
    $(kubectl get pod -l app=orgchart -o jsonpath="{.items[0].metadata.name}") \
    --container vault-agent

## Looking at the secret in the POD (`orgchart` container)

In [ ]:
%%bash
kubectl exec \
    $(kubectl get pod -l app=orgchart -o jsonpath="{.items[0].metadata.name}") \
    --container orgchart -- cat /vault/secrets/database-config.txt

### Redeploy Deployment using `templating`

In [ ]:
%%bash
cat > $WORKDIR/deployment-orgchart.yaml <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: orgchart
  labels:
    app: orgchart
spec:
  selector:
    matchLabels:
      app: orgchart
  replicas: 1
  template:
    metadata:
      annotations:
        vault.hashicorp.com/agent-inject: 'true'
        vault.hashicorp.com/agent-inject-status: 'update'
        vault.hashicorp.com/role: 'internal-app'
        vault.hashicorp.com/agent-inject-secret-database-config.txt: 'internal/data/database/config'
        vault.hashicorp.com/tls-skip-verify: 'true'
        vault.hashicorp.com/agent-inject-template-database-config.txt: |
          {{- with secret "internal/data/database/config" -}}
          postgresql://{{ .Data.data.username }}:{{ .Data.data.password }}@postgres:5432/wizard
          {{- end -}}
      labels:
        app: orgchart
    spec:
      serviceAccountName: internal-app
      containers:
        - name: orgchart
          image: nginx
EOF
kubectl delete --filename $WORKDIR/deployment-orgchart.yaml
kubectl apply --filename $WORKDIR/deployment-orgchart.yaml

### Looking at the new secret formatted

In [ ]:
%%bash
kubectl exec \
    $(kubectl get pod -l app=orgchart -o jsonpath="{.items[0].metadata.name}") \
    -c orgchart -- cat /vault/secrets/database-config.txt

# Clean UP

In [ ]:
# In terminal
vault auth disable k8snorth/ 
vault secrets disable internal

In [ ]:
! kubectl delete --filename $WORKDIR/deployment-orgchart.yaml